In [23]:
import faiss

index = faiss.read_index("index/wikipedia_202307.index")


In [24]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000026448181020> >

In [2]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("CLARIN_API_KEY")

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"

def get_models():
    headers = {
        'Authorization': f'Bearer {API_KEY}',  
        'Content-Type': 'application/json'
    }
    response = requests.get(MODELS_ENDPOINT, headers=headers)  
    if response.status_code == 200:
        return response.json()  

models = get_models()
print(models)


{'data': [{'id': 'bielik', 'full_name': 'speakleash/Bielik-11B-v2.2-Instruct', 'name': 'speakleash/Bielik-11B-v2.2-Instruct'}, {'id': 'cohere', 'full_name': 'CohereForAI/c4ai-command-r-plus', 'name': 'CohereForAI/c4ai-command-r-plus'}, {'id': 'mixtral-8x22B', 'full_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'name': 'mistralai/Mixtral-8x22B-Instruct-v0.1'}, {'id': 'llama3.1-8b', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama-guard', 'full_name': 'meta-llama/Llama-Guard-3-8B', 'name': 'meta-llama/Llama-Guard-3-8B'}, {'id': 'llama3.1', 'full_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'name': 'meta-llama/Meta-Llama-3.1-70B-Instruct'}, {'id': 'openchat', 'full_name': 'openchat/openchat-3.5-1210', 'name': 'openchat/openchat-3.5-1210'}, {'id': 'pllum-shparag-llama3-8B', 'full_name': 'pllum/rag_70k_nemotron_finetun

In [ ]:
def generate_response_from_faiss(query, faiss_results):
    # 1. Przygotuj kontekst na podstawie wyników FAISS
    context = "\n".join([str(result) for result in faiss_results])  # Zmienna `result` to fragmenty tekstów z FAISS
    
    # 2. Przygotuj zapytanie do modelu
    prompt = f"Given the following context, answer the question: {query}\n\nContext:\n{context}"
    
    # 3. Nagłówki do zapytania
    headers = {
        'Authorization': f'Bearer {API_KEY}',  # Twój klucz API CLARIN
        'Content-Type': 'application/json'
    }
    
    # 4. Przygotowanie danych do wysłania
    data = {
        "model": "meta-llama/Llama-3.1-8B-Instruct",  # Wybrany model
        "messages": [{"role": "user", "content": prompt}]
    }
    
    response = requests.post(COMPLETIONS_ENDPOINT, headers=headers, json=data)
    
    if response.status_code == 200:
        response_data = response.json()
        return response_data['choices'][0]['message']['content']
    else:
        print(f"Failed to generate response: {response.status_code}, {response.text}")
        return None

In [10]:
import requests
from faiss import read_index
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS as LangChainFAISS

# Załaduj indeks FAISS
index_path = "index/wikipedia_202307.index"
sentence_index = read_index(index_path)

# Ustawienia API CLARIN
api_key = API_KEY
base_url = "https://services.clarin-pl.eu"


In [ ]:
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer

load_dotenv()

API_KEY = os.getenv("CLARIN_API_KEY")

# Ścieżka do pliku z indeksem FAISS
index_path = "index/wikipedia_202307.index"

# Inicjalizacja modelu do zamiany tekstu na wektory
model = SentenceTransformer('all-MiniLM-L6-v2')

# Ładowanie indeksu FAISS
def load_faiss_index(index_path):
    index = faiss.read_index(index_path)
    return index

# Funkcja do pobierania tytułów artykułów z indeksu FAISS
def get_titles_from_faiss_index(index, k=5):
    # Pobierz tytuły artykułów zapisane w FAISS w formie stringów (zakładając, że tytuły są zapisane w docstrings)
    # Pierwszym krokiem jest przekonwertowanie numerów indeksów na teksty
    titles = []
    for i in range(k):  # Załóżmy, że przeszukamy 5 wyników
        # Wczytaj tytuły z wyników
        title = f"Article {i}"  # W miejsce 'Article {i}' powinny pojawić się odpowiednie tytuły zapisane w FAISS
        titles.append(title)
    return titles

# Konwersja tekstu na wektor
def text_to_vector(text):
    return model.encode([text])[0]

# Wyszukiwanie podobnych dokumentów w indeksie FAISS
def retrieve_similar_documents(query, index, k=5):
    query_vector = text_to_vector(query)  # Przekształć zapytanie na wektor
    query_vector = np.array([query_vector]).astype(np.float32)  # Faiss wymaga typu np.float32
    distances, indices = index.search(query_vector, k)  # k to liczba najbliższych sąsiadów
    return indices, distances

# Funkcja do pobierania pełnych artykułów z Wikipedii na podstawie tytułów
def get_wikipedia_article(title):
    url = f"https://pl.wikipedia.org/w/api.php?action=query&format=json&prop=extracts&exintro&titles={title}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pages = data.get("query", {}).get("pages", {})
        page_id = list(pages.keys())[0]
        extract = pages[page_id].get("extract", "No extract available.")
        return extract
    else:
        return "Error fetching article."

# Funkcja do generowania odpowiedzi z API CLARIN
def generate_answer_from_clarin(query, context):
    url = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
    
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "pllum/rag_70k_nemotron_finetune_pol_llama8b",  # Użycie wybranego modelu
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query},
            {"role": "assistant", "content": context}
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        response_data = response.json()
        return response_data.get("choices", [{}])[0].get("message", {}).get("content", "No answer found")
    else:
        return f"Error: {response.status_code}"

# Funkcja do pobrania dokumentów z Wikipedii na podstawie wyników FAISS
def get_documents_from_indices(indices, index, k=5):
    titles = get_titles_from_faiss_index(index, k)  # Pobieramy tytuły z indeksu
    documents = []
    for idx in indices[0]:  # Zakładamy, że wektory są jednowymiarowe (rozmiar k, 1)
        title = titles[idx]  # Mapowanie indeksu na tytuł (prosty sposób)
        article = get_wikipedia_article(title)
        documents.append(f"Title: {title}\n{article}\n")
    
    return "\n".join(documents)

# Główna funkcja do obsługi procesu RAG
def rag_system(query):
    # Załaduj indeks FAISS
    index = load_faiss_index(index_path)
    
    # Wyszukaj najbardziej podobne dokumenty w FAISS
    indices, distances = retrieve_similar_documents(query, index)

    # Pobierz dokumenty z Wikipedii na podstawie wyników
    context = get_documents_from_indices(indices, index)

    # Wygeneruj odpowiedź z CLARIN na podstawie kontekstu
    answer = generate_answer_from_clarin(query, context)
    
    return answer

# Przykład użycia systemu RAG
query = "Co to jest sztuczna inteligencja?"
answer = rag_system(query)
print(answer)


Response from CLARIN API: {'detail': "Non existing model 'meta-llama/Llama-3.1-8B-Instruct'"}
Error: The response from the API does not contain 'choices'.


______________

In [6]:
import os
import requests
import numpy as np
from faiss import read_index
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

# Załaduj zmienne środowiskowe z pliku .env
load_dotenv()

# Odczytaj klucz API CLARIN z pliku .env
API_KEY = os.getenv("CLARIN_API_KEY")

# Endpointy API CLARIN
MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"

# Endpoint API Wikipedii
WIKIPEDIA_API_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# Ścieżka do indeksu FAISS
index_path = "index/wikipedia_202307.index"  # Zaktualizuj ścieżkę do pliku .index

# Załaduj model SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Załaduj indeks FAISS
index = read_index(index_path)

# Funkcja do generowania embeddingu zapytania
def get_embedding(query):
    # Generowanie embeddingu dla zapytania
    embedding = model.encode(query)
    return np.array([embedding]).astype(np.float32)  # Konwersja na odpowiedni format FAISS

# Funkcja do wyszukiwania podobnych artykułów w FAISS
def search_faiss(query_embedding, k=5):
    _, indices = index.search(query_embedding, k)  # k to liczba wyników
    return indices

# Funkcja do pobierania pełnych artykułów z Wikipedii
def get_wikipedia_article(title):
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "exintro": True,  # Pobieramy tylko wstęp artykułu
        "explaintext": True
    }
    response = requests.get(WIKIPEDIA_API_ENDPOINT, params=params)
    data = response.json()
    
    # Zwróć pełny artykuł, jeśli został znaleziony
    pages = data["query"]["pages"]
    page_id = next(iter(pages))  # Pobierz pierwszy (jedyny) artykuł
    article_text = pages[page_id].get("extract", "No content found")
    
    return article_text

# Funkcja do generowania odpowiedzi na podstawie wyników FAISS i modelu CLARIN
def generate_response_from_faiss(query, faiss_results):
    # Przygotuj kontekst na podstawie wyników FAISS
    context = "\n".join([str(result) for result in faiss_results])  # Zmienna `result` to fragmenty tekstów z FAISS
    
    # Przygotuj zapytanie do modelu
    prompt = f"Given the following context, answer the question: {query}\n\nContext:\n{context}"
    
    # Nagłówki do zapytania
    headers = {
        'Authorization': f'Bearer {API_KEY}',  # Twój klucz API CLARIN
        'Content-Type': 'application/json'
    }
    
    # Przygotowanie danych do wysłania
    data = {
        "model": 'speakleash/Bielik-11B-v2.2-Instruct',  # Wybrany model
        "messages": [{"role": "user", "content": prompt}]
    }
    
    # Wysyłanie zapytania do modelu
    response = requests.post(COMPLETIONS_ENDPOINT, headers=headers, json=data)
    
    if response.status_code == 200:
        response_data = response.json()
        return response_data['choices'][0]['message']['content']
    else:
        print(f"Failed to generate response: {response.status_code}, {response.text}")
        return None

# Funkcja do integracji RAG (Retrieval-Augmented Generation) za pomocą LangChain
def run_rag_chain(query):
    # Generowanie embeddingu zapytania
    query_embedding = get_embedding(query)

    # Wyszukiwanie podobnych artykułów w FAISS
    indices = search_faiss(query_embedding)
    
    # Zakładając, że 'indices' zawierają numery indeksów w FAISS, które reprezentują tytuły artykułów
    # Przykładowe tytuły artykułów (należy zamienić je na rzeczywiste tytuły na podstawie wyników FAISS)
    faiss_titles = []  # Lista tytułów, które odpowiadają indeksom FAISS
    
    for idx in indices[0]:  # Zakładając, że indices[0] zawiera odpowiednie numery indeksów
        faiss_titles.append(f"Article_{idx}")  # Proszę zmienić w zależności od struktury FAISS
    
    # Pobierz pełne artykuły z Wikipedii
    articles = [get_wikipedia_article(title) for title in faiss_titles]

    # Wygenerowanie odpowiedzi z modelu CLARIN
    response = generate_response_from_faiss(query, articles)

    return response

# Testowanie zapytania
query = "What are the applications of AI?"  # Przykładowe zapytanie
response = run_rag_chain(query)

print("Response from model:", response)


Failed to generate response: 404, {"detail":"Non existing model 'speakleash/Bielik-11B-v2.2-Instruct'"}
Response from model: None


In [26]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Załaduj plik indeksu FAISS
index = faiss.read_index(r"C:\Users\kjani\nlp\NLP\task_3\index\wikipedia_202307.index")

# Sprawdź informacje o wymiarze indeksu
print("Wymiar wektora:", index.d)


Wymiar wektora: 384


In [27]:
# Załaduj model do przetwarzania zapytań
model = SentenceTransformer('all-MiniLM-L6-v2')  # Używamy tego samego modelu, co do tworzenia wektorów

# Funkcja przekształcająca tekst w wektor
def get_query_embedding(query):
    return model.encode([query])

# Przykład zapytania
query = "Sztuczna inteligencja"

# Przekształcenie zapytania na wektor
query_embedding = np.array(get_query_embedding(query), dtype=np.float32)


In [28]:
# Wyszukaj k najbliższych sąsiedztw w indeksie FAISS
k = 5  # Liczba wyników
distances, indices = index.search(query_embedding, k)

# Wyświetl wyniki
print("Podobne artykuły:")
for i in range(k):
    print(f"Indeks {indices[0][i]} - Odległość: {distances[0][i]}")


Podobne artykuły:
Indeks 261157 - Odległość: 0.7854613065719604
Indeks 5485593 - Odległość: 0.7977433204650879
Indeks 5490606 - Odległość: 0.8000825643539429
Indeks 5485115 - Odległość: 0.8073369860649109
Indeks 261189 - Odległość: 0.8139570355415344


In [41]:
import wikipediaapi

# Definiujemy niestandardowy User-Agent zgodny z polityką Wikipedii
user_agent = "YourAppName/1.0 (https://example.com; contact@example.com)"

# Tworzymy obiekt Wikipedia z niestandardowym User-Agent
wiki = wikipediaapi.Wikipedia(
    user_agent=user_agent,  # Dodajemy User-Agent jako pierwszy argument
    language='pl'           # Ustawiamy język na polski
)

# Funkcja do pobierania tytułu artykułu na podstawie tytułu
def get_article_intro(title):
    page = wiki.page(title)
    if page.exists():
        return title, page.text.split('\n')[0]  # Zwraca tytuł i pierwsze zdanie
    return None

# Przykład użycia
article_title, first_sentence = get_article_intro("Borys Szyc")
print(article_title, first_sentence)


Borys Szyc Borys Michał Szyc (ur. 4 września 1978 w Łodzi jako Borys Michał Michalak) – polski aktor teatralny i filmowy, piosenkarz oraz lektor.


In [44]:
import requests
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import wikipediaapi

CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"
USER_AGENT = "YourAppName/1.0 (https://example.com; contact@example.com)"  # Wikipedia User-Agent

# Tworzymy obiekt Wikipedii
wiki = wikipediaapi.Wikipedia(user_agent=USER_AGENT, language='pl')


# Model Sentence-Transformers (ten sam użyty do generowania indeksu)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Funkcja do wyszukiwania w FAISS
def search_faiss(query, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(query_embedding, k)
    return distances[0], indices[0]

# Pobieranie artykułów z Wikipedii
def get_article_intro(title):
    page = wiki.page(title)
    if page.exists():
        return title, page.text.split('\n')[0]  # Zwróć tytuł i pierwsze zdanie
    return None

# Funkcja do wysyłania zapytania do modelu CLARIN
def clarin_chat_completion(model_id, prompt, max_tokens=150):
    url = f"{CLARIN_BASE_URL}/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

# Przykład: pełny przepływ wyszukiwania i odpowiedzi
def handle_query_with_rag(user_query, model_id="bielik", k=5):
    # 1. Znajdź artykuły w indeksie FAISS
    distances, indices = search_faiss(user_query, k)
    
    # 2. Pobierz tytuły i wstępy artykułów
    articles = []
    for idx in indices:
        # FAISS indeksuje tytuły; zakładamy, że tytuły odpowiadają numerom indeksu
        title = f"Article_{idx}"  # Zakładamy istnienie tytułu dla indeksu
        intro = get_article_intro(title)
        if intro:
            articles.append(intro)

    # 3. Przygotuj kontekst do zapytania CLARIN
    context = "\n\n".join([f"{title}: {intro}" for title, intro in articles])
    prompt = f"Zapytanie: {user_query}\n\nKontekst:\n{context}\n\nOdpowiedz w kontekście powyższych informacji."

    # 4. Wyślij zapytanie do modelu CLARIN
    response = clarin_chat_completion(model_id, prompt)
    return response


In [45]:
try:
    user_query = "Gdzie urodziła sie Edyta Górniak?"
    response = handle_query_with_rag(user_query)
    print("Odpowiedź modelu CLARIN:")
    print(response)
except Exception as e:
    print(f"Error: {e}")

Odpowiedź modelu CLARIN:
Edyta Górniak urodziła się dnia 14 listopada 1972 roku w Ziębicach na Dolnym Śląsku, w województwie dolnośląskim.

### Kontekst:

Jak widać, pytanie dotyczy miejsca urodzenia artystki o imieniu i nazwisku Edyta Górniak. Powyższe informacje potwierdzają, że Edyta Górniak przyszła na świat w mieście Ziębice, znajdującym się na Dolnym Śląsk


______________